In [1]:
require 'torch'
require 'nn'
require 'image'
require 'optim'

In [2]:
loadcaffe_wrap = require 'loadcaffe_wrapper'
cjson = require 'cjson'
string = require 'string'

In [3]:
cmd = torch.CmdLine()

-- Basic options
cmd:option('-style_dir', 'data/cubism/', 'Style input directory')
cmd:option('-tmp_dir', 'tmp/', 'Directory to store vectors on disk')
cmd:option('-gpu', -1, 'Zero-indexed ID of the GPU to use; for CPU mode set -gpu = -1')

-- Other options
cmd:option('-pooling', 'max', 'max|avg')
cmd:option('-proto_file', 'models/VGG_ILSVRC_19_layers_deploy.prototxt')
cmd:option('-model_file', 'models/VGG_ILSVRC_19_layers.caffemodel')

cmd:option('-content_layers', 'relu4_2', 'layers for content')
cmd:option('-style_layers', 'relu4_1', 'layers for style') -- tbh all but relu6 and relu7, which cause size mismatches
                            -- 'relu1_1,relu2_1,relu3_1,relu4_1,relu5_1'
                            -- 'relu1_1,relu1_2,relu2_1,relu2_2,relu3_1,relu3_2,relu3_3,relu3_4,relu4_1,relu4_2,relu4_3,relu4_4,relu5_1,relu5_2,relu5_3,relu5_4'

In [4]:
-- Preprocess an image before passing it to a Caffe model.
-- We need to rescale from [0, 1] to [0, 255], convert from RGB to BGR,
-- and subtract the mean pixel. [jcjohnson]
function preprocess(img)
  local mean_pixel = torch.DoubleTensor({103.939, 116.779, 123.68})
  local perm = torch.LongTensor{3, 2, 1}
  img = img:index(1, perm):mul(256.0)
  mean_pixel = mean_pixel:view(3, 1, 1):expandAs(img)
  img:add(-1, mean_pixel)
  return img
end

In [5]:
-- Returns a network that computes the CxC Gram matrix from inputs
-- of size C x H x W – jcjohnson's version
function GramMatrix()
    local net = nn.Sequential()
    net:add(nn.View(-1):setNumInputDims(2))
    local concat = nn.ConcatTable()
    concat:add(nn.Identity())
    concat:add(nn.Identity())
    net:add(concat)
    net:add(nn.MM(false, true))
    return net
end

In [6]:
-- utility function to reshape a tensor from M x N x ... to an MxN array
function flatten(t)
    return torch.view(t, -1) -- :storage() exposes a raw memory interface
end

In [7]:
function Style2Vec(img, cnn)
    --[[ runs img through cnn, saving the output tensor at each of style_layers

    relu1_1 : FloatTensor - size: 64x64
    relu1_2 : FloatTensor - size: 64x64
    relu2_1 : FloatTensor - size: 128x128
    relu2_2 : FloatTensor - size: 128x128
    relu3_1 : FloatTensor - size: 256x256
    relu3_2 : FloatTensor - size: 256x256
    relu3_3 : FloatTensor - size: 256x256
    relu3_4 : FloatTensor - size: 256x256
    relu4_1 : FloatTensor - size: 512x512
    relu4_2 : FloatTensor - size: 512x512
    relu4_3 : FloatTensor - size: 512x512
    relu4_4 : FloatTensor - size: 512x512
    relu5_1 : FloatTensor - size: 512x512
    relu5_2 : FloatTensor - size: 512x512
    relu5_3 : FloatTensor - size: 512x512
    relu5_4 : FloatTensor - size: 512x512
    
    Returns a Lua table with the above key-value pairs.
    
    --]]
    
    local next_style_idx = 1
    local net = nn.Sequential()
    local style_vec = {}
    local style_layers = params.style_layers:split(',')
    
    -- Build up net from cnn
    
    for i = 1, #cnn do
        if next_style_idx <= #style_layers then
            local layer = cnn:get(i)
            local layer_name = layer.name
            local layer_type = torch.type(layer)
            local is_pooling = (layer_type == 'cudnn.SpatialMaxPooling' or layer_type == 'nn.SpatialMaxPooling')
            
            -- add layers to net from cnn, replacing max-pooling if necessary [jcjohnson]
            if is_pooling and params.pooling == 'avg' then
                local msg = 'Replacing max pooling at layer %d with average pooling'
                print(string.format(msg, i))
                assert(layer.padW == 0 and layer.padH == 0)
                -- kWxkH regions by step size dWxdH
                local kW, kH = layer.kW, layer.kH
                local dW, dH = layer.dW, layer.dH
                local avg_pool_layer = nn.SpatialAveragePooling(kW, kH, dW, dH):float()
                if params.gpu >= 0 then avg_pool_layer:cuda() end
                net:add(avg_pool_layer)
            else
                net:add(layer)
            end
            
            -- now to grab style layers
            
            if (layer_name == style_layers[next_style_idx]) then
                    
                local gram = GramMatrix():float()
                if params.gpu >= 0 then gram = gram:cuda() end
                local target_features = net:forward(img)
                local target_i = gram:forward(target_features)
                
                target_i:div(target_features:nElement())

                style_vec[layer_name] = torch.totable(target_i)
                -- itorch.image(target_i) -- YA THIS IS THE VECTOR!!!
                                
                next_style_idx = next_style_idx + 1
            end
        end
    end
        
    return style_vec
end

In [8]:
function load_json(filename, file)
    
    local str = torch.load(params.tmp_dir .. filename .. '.json', 'ascii')
    
    -- print('no copy of ' .. filename .. ' exists')
    
    return cjson.decode(str)
end

In [ ]:
function cache_json(filename, file)    
    if paths.dir(tmp) == nil then paths.mkdir(tmp) end
    
    json_string = cjson.encode(file)
    torch.save(params.tmp_dir .. filename .. '.json', json_string, 'ascii')
end

In [10]:
function cached(label) -- is it cached? t/f
    for f in paths.iterfiles(params.tmp_dir) do
        if f == filename then
            -- print(filename .. 'already exists')
            return true
        end
    end
    return false
end

In [11]:
-- let's get started

local arg = {} -- when running from cli, this will be defined
params = cmd:parse(arg)
print(params)

{
  gpu : -1
  pooling : max
  content_layers : relu4_2
  style_dir : data/cubism/
  style_layers : relu4_1
  tmp_dir : tmp/
  proto_file : models/VGG_ILSVRC_19_layers_deploy.prototxt
  model_file : models/VGG_ILSVRC_19_layers.caffemodel
}


In [12]:
-- load caffe network image
cnn = loadcaffe_wrap.load(params.proto_file, params.model_file, params.backend):float()

if params.gpu >= 0 then cnn:cuda() end

Successfully loaded models/VGG_ILSVRC_19_layers.caffemodel


conv1_1: 64 3 3 3


conv1_2: 64 64 3 3
conv2_1: 128 64 3 3
conv2_2: 128 128 3 3
conv3_1: 256 128 3 3
conv3_2: 256 256 3 3
conv3_3: 256 256 3 3
conv3_4: 256 256 3 3


conv4_1: 512 256 3 3


conv4_2: 512 512 3 3


...

conv5_3: 512 512 3 3


conv5_4: 512 512 3 3


fc6: 1 1 25088 4096


fc7: 1 1 4096 4096


fc8: 1 1 4096 1000


In [14]:
-- load style_images

style_images = {}

for f in paths.iterfiles(params.style_dir) do
    if string.match(f, '.jpg') then

        local img = image.load(params.style_dir .. f)
        img = preprocess(img):float()

        if params.gpu >= 0 then img = img:cuda() end

        label = string.split(f, '.jpg')[1]

        -- itorch.image(img)
        style_images[label] = img
    end
end

print(style_images)

{
  6_ARTSTOR_103_41822003848650 : FloatTensor - size: 3x768x596
  7_AMICO_METRO_103826558 : FloatTensor - size: 3x768x633
  18_ARTSTOR_103_41822003848767 : FloatTensor - size: 3x768x559
  22_ARTSTOR_103_41822003848817 : FloatTensor - size: 3x748x1024
  3_AMOMA_10312309950 : FloatTensor - size: 3x768x795
  13_ARTSTOR_103_41822003848718 : FloatTensor - size: 3x768x590
  9_ARTSTOR_103_41822001700788 : FloatTensor - size: 3x272x400
  2_ARTSTOR_103_41822003778188 : FloatTensor - size: 3x768x571
  19_ARTSTOR_103_41822003848759 : FloatTensor - size: 3x768x569
  10_ARTSTOR_103_41822003288345 : FloatTensor - size: 3x768x569
  8_ASFMOMAIG_10312704327 : FloatTensor - size: 3x768x599
  12_ARTSTOR_103_41822003848734 : FloatTensor - size: 3x768x580
  14_ARTSTOR_103_41822003848700 : FloatTensor - size: 3x768x666
  20_ARTSTOR_103_41822003848791 : FloatTensor - size: 3x768x574
  16_ARTSTOR_103_41822003848783 : FloatTensor - size: 3x768x605
  5_ARTSTOR_103_41822003755939 : FloatTensor - size: 3x673x102

In [15]:
collectgarbage('count')

2348.5419921875	


In [ ]:
-- Run Style2Vec, caching tensors in tmp/

-- styles = {}
for label, image in pairs(style_images) do

    io.write(label .. ':\t' .. params.style_layers .. ' ...' ) 
    
    local wrapper = {}
    
    if not cached(label) then
        
        local vec = Style2Vec(image, cnn)
        wrapper['image'] = image
        wrapper['style'] = vec
        
        io.write(' caching ... ')
        cache_json(label, wrapper)
    else
        wrapper = load_json(label) 
    end
    
--     styles[label] = wrapper

    io.write(' Done!\n')
        
    collectgarbage()
end

6_ARTSTOR_103_41822003848650:	relu4_1 ...

In [ ]:
print(#styles)

In [ ]:
-- clean up a little
cnn = nil
style_images = nil
collectgarbage()

In [ ]:
collectgarbage('count')

In [39]:
local ct = 1
local size = 262144 -- 512^2 - relu4_1

local tensors = nil
local labels = torch.String

for f in paths.iterfiles(params.tmp_dir) do
    local table = torch.load(params.tmp_dir .. f, params.cache_format)
    t = flatten(table['relu4_1']:double())
    
    if tensors then
        tensors = torch.cat(tensors, t)
    else
        tensors = t
    end
        
    ct = ct + 1    
end

views = torch.view(tensors, -1, size)

print(#views)



     19
 262144
[torch.LongStorage of size 2]



In [35]:
local m = require 'manifold'
p = m.embedding.tsne(views, {dim=2, perplexity=8})

Computing P-values...	
Running t-SNE...	
Iteration 10: KL divergence is 26.355649156069	
Iteration 20: KL divergence is 21.360202021903	
Iteration 30: KL divergence is 23.230125370646	
Iteration 40: KL divergence is 27.024930613218	
Iteration 50: KL divergence is 25.266509976266	
Iteration 60: KL divergence is 26.771714931245	
Iteration 70: KL divergence is 24.595036025478	
Iteration 80: KL divergence is 25.046394745488	
Iteration 90: KL divergence is 27.353937811515	
Iteration 100: KL divergence is 24.049066734159	
Iteration 110: KL divergence is 26.549208571448	
Iteration 120: KL divergence is 23.401027251369	
Iteration 130: KL divergence is 26.38630383098	
Iteration 140: KL divergence is 24.470353374956	
Iteration 150: KL divergence is 32.283046492902	
Iteration 160: KL divergence is 25.86199844962	
Iteration 170: KL divergence is 25.761608615848	
Iteration 180: KL divergence is 24.507163955304	
Iteration 190: KL divergence is 23.887872709282	
Iteration 200: KL divergence is 2.24508

5	
Iteration 830: KL divergence is 0.32074039541963	
Iteration 840: KL divergence is 0.31768209716727	
Iteration 850: KL divergence is 0.31435088638005	
Iteration 860: KL divergence is 0.31180135275276	
Iteration 870: KL divergence is 0.31011885819432	
Iteration 880: KL divergence is 0.30891456891116	
Iteration 890: KL divergence is 0.30792547875231	
Iteration 900: KL divergence is 0.30709504829411	
Iteration 910: KL divergence is 0.30631784318907	
Iteration 920: KL divergence is 0.30506288442605	
Iteration 930: KL divergence is 0.30275314282627	
Iteration 940: KL divergence is 0.29980855761917	
Iteration 950: KL divergence is 0.29689501241048	
Iteration 960: KL divergence is 0.29443203411925	
Iteration 970: KL divergence is 0.29242770326785	
Iteration 980: KL divergence is 0.29070046805961	
Iteration 990: KL divergence is 0.28910741624957	
Iteration 1000: KL divergence is 0.28739712607142	


In [36]:
print(p)

 604.7034 -269.6182
 258.5209   13.2617
-290.1599   -2.6792
 464.2557   59.8725
 -77.4056   73.1160
-511.0287   84.7488
-114.3906 -147.1168
-453.5069  -56.0094
 -42.9370  207.1664
 125.1980 -348.7263
 407.7959 -151.6246
 232.3759 -133.6583
-382.3847  158.9189
 123.6943   35.7421
-305.5637 -288.9049
 289.8681  273.3978
-339.4825  374.2545
  -3.3650 -442.0694
  13.8126  559.9284
[torch.DoubleTensor of size 19x2]



---

In [ ]:
function CosineSimilarity(x, y)
    local net = nn.Sequential()
    net:add(nn.CosineDistance())
    return net:forward({x, y})
end

function StyleDistance(x, y, sorted_layers)
    -- this function will return the distance from each layer, assuming x and y
    -- x["relu2_1 "] = torch.FloatTensor
    
    for _, i in ipairs(sorted_layers) do -- can you tell I'm new to Lua?
        local distance_vector = CosineSimilarity(x[i]:double(), y[i]:double())
        local avg_distance = torch.mean(distance_vector)
        
        local msg ='Distance at layer %s is: %f'
        print(string.format(msg, i, avg_distance))
    end
    
end

In [ ]:

-- -- this is a little embarassing, no?
-- local labels = params.style_layers:split(',')
-- table.sort(labels)

StyleDistance(style_vecs['haring_bw.jpg'], style_vecs['haring_bw.jpg'], labels)
-- -- x = torch.Tensor({1, 2, 3})
-- -- y = torch.Tensor({4, 5, 6})
-- -- print(CosineSimilarity(x, y))